# SPIKE cluster analysis

7.04.2023 Sören

## Initial setup

In [ ]:
import sys
from pathlib import Path
import re
import logging
import itertools

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mplcolors
from colorcet import m_gray
from scipy import stats
import boost_histogram as bh
import plotly

import locan as lc

In [ ]:
lc.show_versions(dependencies=False)

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [ ]:
rng = np.random.default_rng(seed=1)

## Load data

In [ ]:
files = lc.Files.from_glob(
    pattern="**/*_AntiS2Spike568*.txt",
    directory=Path('.') / '../data/two-channel',
    column="receptor"
)
stoplist = lc.Files.concatenate([
    lc.Files.from_glob(directory=files.directory, pattern="**/*settings.txt"),
    lc.Files.from_glob(directory=files.directory, pattern="**/*README*.txt"),
    lc.Files.from_glob(directory=files.directory, pattern="**/*/beads + matrix/*.txt")
])
files.exclude(stoplist=stoplist, column="receptor")

other_files = lc.Files.from_glob(
    pattern="**/*_Cy5*.txt",
    directory=files.directory
)
other_files.exclude(stoplist=stoplist)
files.match_files(files=other_files.df, column="receptor", other_column="other")

matrix_files = lc.Files.from_glob(
    pattern="**/*_RAW*.txt",
    directory=files.directory
)
matrix_files.exclude(stoplist=stoplist)
files.match_files(files=matrix_files.df, column="receptor", other_column="matrix")

files.match_file_upstream(column="receptor", pattern="metadata.toml")

files.print_summary()
files.df.applymap(lambda x: x.name)

Set the selector to the index for a single file to be analyzed.

In [ ]:
# This cell has a tag 'parameters'
# It will be modified by a script running this notebook with all listed files.

selector = 0

In [ ]:
print(f'selector: {selector}')
files[selector]

In [ ]:
files[0].apply(lambda x: x.name)

### Prepare metadata

In [ ]:
metadata = lc.load_metadata_from_toml(files[selector].metadata)['metadata']

### Load localization file

In [ ]:
locdatas = [
    lc.load_locdata(files[selector].receptor, file_type=lc.FileType.RAPIDSTORM),
    lc.load_locdata(files[selector].other, file_type=lc.FileType.RAPIDSTORM)
]

for locdata in locdatas:
    locdata.meta.MergeFrom(metadata)
    
locdatas[0].dataframe = locdatas[0].dataframe.assign(channel= 0)
locdatas[1].dataframe = locdatas[1].dataframe.assign(channel= 1)

for locdata in locdatas:
    print(f'Metadata:\n')
    locdata.print_summary()

## Select

In [ ]:
conditions = [
    '1_000 < frame < 25_000  and 5_000 < intensity',
    '1_000 < frame < 25_000 and 5_000 < intensity'
]

for i, (locdata, condition) in enumerate(zip(locdatas, conditions)):
    locdata = lc.select_by_condition(locdata, condition=condition)
    locdata.reduce()
    locdatas[i] = locdata
    print(condition)

In [ ]:
locdata = locdatas[0]
locdata.properties

In [ ]:
locdata.data.head()

In [ ]:
locdatas[0].data.describe()

In [ ]:
locdatas[1].data.describe()

## Transform

### Bunwarp

In [ ]:
matrix_path = files[selector].matrix
assert matrix_path.exists()
matrix_path

In [ ]:
locdatas[0] = lc.bunwarp(locdata=locdatas[0], matrix_path=matrix_path, pixel_size=(10, 10), flip=True)

## Visualize

In [ ]:
figsize = np.array([16, 8]) * 2
bin_size = 50

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
lc.render_2d(locdatas[0], ax=ax, bin_size=bin_size, rescale=lc.Trafo.EQUALIZE);

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
lc.render_2d(locdatas[1], ax=ax, bin_size=bin_size, rescale=lc.Trafo.EQUALIZE);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=figsize)
for locdata, ax in zip(locdatas, ax):
    lc.render_2d(locdata, ax=ax, bin_size=bin_size, rescale=lc.Trafo.EQUALIZE_0P3);
    # locdata.region.plot(ax=ax, fill=False, color='White');
    ax.set(aspect='equal')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=figsize)
for locdata, ax in zip(locdatas, ax):
    lc.render_2d_mpl(locdata, ax=ax, bin_size=100, rescale=lc.Trafo.NONE, vmin=0, vmax=500);
    # locdata.region.plot(ax=ax, fill=False, color='White');
    ax.set(aspect='equal')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
lc.render_2d_rgb_mpl(locdatas, bin_size=bin_size, rescale=lc.Trafo.EQUALIZE);

## Cluster combined data

In [ ]:
locdata_all = lc.LocData.concat(locdatas)
locdata_all.properties

In [ ]:
locdata_all.data.describe()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
lc.render_2d_mpl(locdata_all, bin_size=bin_size, rescale=lc.Trafo.EQUALIZE);

### Cluster data

In [ ]:
%%time
noise, clust = lc.cluster_dbscan(locdata_all, eps=75, min_samples=20)

In [ ]:
n_clustered_loc = np.sum([ref.properties['localization_count'] for ref in clust.references])
print(f"Number of clusters: {clust.properties['localization_count']}")
print(f"Number of noise localizations: {noise.properties['localization_count']}")
print(f"Number of clustered localizations: {n_clustered_loc}")
print(f"Ratio cluster to noise localizations: {n_clustered_loc / (n_clustered_loc + noise.properties['localization_count']):.3}")

In [ ]:
clust.properties

### Compute convex hull for each cluster.

In [ ]:
clust.update_convex_hulls_in_references()

Optional: Turn nan entries that arise from computation of region measures for non-existing hull objects to zero. This allows plotting histograms for region measures and similar properties.

## Cluster properties (all cluster)

### Number of localizations per cluster

In [ ]:
clust_prop_nloc = lc.LocalizationProperty(loc_property='localization_count').compute(clust)

In [ ]:
clust_prop_nloc.hist(bins=100);

In [ ]:
clust_prop_nloc.distribution_statistics.parameter_dict()

### Cluster areas as estimated by the convex hull of all localizations in the cluster

In [ ]:
clust_prop_region = lc.LocalizationProperty(loc_property='region_measure_ch').compute(clust)

In [ ]:
clust_prop_region.hist(bins=100);

In [ ]:
clust_prop_region.distribution_statistics.parameter_dict()

### Inertia Moments

In [ ]:
clust.update_inertia_moments_in_references()

In [ ]:
plt.hist(clust.data.circularity_im);

In [ ]:
plt.hist(clust.data.orientation_im);

#### Correlation between localisations per cluster and circularity

In [ ]:
locdata_properties = ['localization_count', 'circularity_im']
axes = bh.axis.AxesTuple((
    bh.axis.Regular(50, 10, 10_000, transform=bh.axis.transform.log), 
    bh.axis.Regular(50, 0, 1)))

histogram = bh.Histogram(*axes)

histogram.reset()
histogram.fill(*clust.data[locdata_properties].values.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
mesh = ax.pcolormesh(*histogram.axes.edges.T, histogram.view().T);
fig.colorbar(mesh);
ax.set(
    xlabel='localization_count',
    ylabel='circularity_im',
    # xticks=histogram.axes[0].edges.T,
    # yticks=histogram.axes[1].edges.T,
    xscale='log',
    yscale='linear'
      );

### Oriented bounding box

In [ ]:
clust.update_oriented_bounding_box_in_references()

In [ ]:
plt.hist(clust.data.circularity_obb);

In [ ]:
plt.hist(clust.data.orientation_obb);

#### Correlation between localisations per cluster and circularity

In [ ]:
locdata_properties = ['localization_count', 'circularity_obb']
axes = bh.axis.AxesTuple((
    bh.axis.Regular(50, 10, 10_000, transform=bh.axis.transform.log), 
    bh.axis.Regular(50, 0, 1)))

histogram = bh.Histogram(*axes)

histogram.reset()
histogram.fill(*clust.data[locdata_properties].values.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
mesh = ax.pcolormesh(*histogram.axes.edges.T, histogram.view().T);
fig.colorbar(mesh);
ax.set(
    xlabel='localization_count',
    ylabel='circularity_im',
    # xticks=histogram.axes[0].edges.T,
    # yticks=histogram.axes[1].edges.T,
    xscale='log',
    yscale='linear'
      );

#### Correlation between circularity_obb and circularity_im

In [ ]:
locdata_properties = ['circularity_obb', 'circularity_im']
axes = bh.axis.AxesTuple((
    bh.axis.Regular(100, 0, 1), 
    bh.axis.Regular(100, 0, 1)))

histogram = bh.Histogram(*axes)

histogram.reset()
histogram.fill(*clust.data[locdata_properties].values.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
mesh = ax.pcolormesh(*histogram.axes.edges.T, histogram.view().T);
fig.colorbar(mesh);
ax.set(
    xlabel='circularity_obb',
    ylabel='circularity_im',
    # xticks=histogram.axes[0].edges.T,
    # yticks=histogram.axes[1].edges.T,
    xscale='linear',
    yscale='linear'
      );

## Select cluster

In [ ]:
clust_selection = clust

### Eliminate cluster on boundary

In [ ]:
clust_selection = lc.select_by_region(clust_selection, region=locdata_all.bounding_box.region.buffer(-200), reduce=False)

### Select by cluster properties

In [ ]:
condition = '0 < localization_count < 10_000 and ' \
            '0.7 < circularity_im and ' \
            '0.4 < circularity_obb and ' \
            '25_000 < region_measure_ch < 60_000'
clust_selection = lc.select_by_condition(clust_selection, condition=condition)

references_ = [clust.references[i] for i in clust_selection.indices]
clust_selection.reduce()
clust_selection.references = references_

In [ ]:
clust_selection.properties

In [ ]:
clust_selection.data.describe()

### Cluster areas as estimated by the convex hull of all localizations in the cluster

In [ ]:
clust_prop_region = lc.LocalizationProperty(loc_property='region_measure_ch').compute(clust_selection)

In [ ]:
clust_prop_region.hist(bins=100);

In [ ]:
clust_prop_region.distribution_statistics.parameter_dict()

### Nearest-neighbor distributions of cluster centroids

In [ ]:
nn = lc.NearestNeighborDistances().compute(clust_selection)

In [ ]:
nn.hist()

In [ ]:
nn.results.index = clust_selection.data.index
clust_selection.dataframe = clust_selection.dataframe.assign(nn_distance=nn.results['nn_distance'])

In [ ]:
colors = clust_selection.data['nn_distance']
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
mappable = ax.scatter(x=clust_selection.coordinates[:,0], y=clust_selection.coordinates[:,1], marker='.', c=colors, vmin=0, vmax=500, label='centroids red');
fig.colorbar(mappable, ax=ax);
ax.set(
    aspect='equal',
    # xlim=(-30, 30),
    # ylim=(-30, 30)
);

## Shape and Orientation

### Inertia Moments

In [ ]:
plt.hist(clust_selection.data.circularity_im);

In [ ]:
plt.hist(clust_selection.data.orientation_im);

#### Correlation between localisations per cluster and circularity

In [ ]:
locdata_properties = ['localization_count', 'circularity_im']
axes = bh.axis.AxesTuple((
    bh.axis.Regular(50, 10, 10_000, transform=bh.axis.transform.log), 
    bh.axis.Regular(50, 0, 1)))

histogram = bh.Histogram(*axes)

histogram.reset()
histogram.fill(*clust_selection.data[locdata_properties].values.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
mesh = ax.pcolormesh(*histogram.axes.edges.T, histogram.view().T);
fig.colorbar(mesh);
ax.set(
    xlabel='localization_count',
    ylabel='circularity_im',
    # xticks=histogram.axes[0].edges.T,
    # yticks=histogram.axes[1].edges.T,
    xscale='log',
    yscale='linear'
      );

### Oriented bounding box

In [ ]:
plt.hist(clust_selection.data.circularity_obb);

In [ ]:
plt.hist(clust_selection.data.orientation_obb);

#### Correlation between localisations per cluster and circularity

In [ ]:
locdata_properties = ['localization_count', 'circularity_obb']
axes = bh.axis.AxesTuple((
    bh.axis.Regular(50, 10, 10_000, transform=bh.axis.transform.log), 
    bh.axis.Regular(50, 0, 1)))

histogram = bh.Histogram(*axes)

histogram.reset()
histogram.fill(*clust_selection.data[locdata_properties].values.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
mesh = ax.pcolormesh(*histogram.axes.edges.T, histogram.view().T);
fig.colorbar(mesh);
ax.set(
    xlabel='localization_count',
    ylabel='circularity_obb',
    # xticks=histogram.axes[0].edges.T,
    # yticks=histogram.axes[1].edges.T,
    xscale='log',
    yscale='linear'
      );

#### Correlation between circularity_obb and circularity_im

In [ ]:
locdata_properties = ['circularity_obb', 'circularity_im']
axes = bh.axis.AxesTuple((
    bh.axis.Regular(100, 0, 1), 
    bh.axis.Regular(100, 0, 1)))

histogram = bh.Histogram(*axes)

histogram.reset()
histogram.fill(*clust_selection.data[locdata_properties].values.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
mesh = ax.pcolormesh(*histogram.axes.edges.T, histogram.view().T);
fig.colorbar(mesh);
ax.set(
    xlabel='circularity_obb',
    ylabel='circularity_im',
    # xticks=histogram.axes[0].edges.T,
    # yticks=histogram.axes[1].edges.T,
    xscale='linear',
    yscale='linear'
      );

## Identify cluster on individual channels

In [ ]:
logger.setLevel(logging.ERROR)
selections_0 = []
selections_1 = []
for reference in clust_selection.references:
    selection = lc.select_by_condition(reference, condition="channel==0")
    selection.region = reference.convex_hull.region
    selections_0.append(selection)
    
    selection = lc.select_by_condition(reference, condition="channel==1")
    selection.region = reference.convex_hull.region
    selections_1.append(selection)
    
clusts = [
    lc.LocData.from_collection(selections_0),
    lc.LocData.from_collection(selections_1)
]
logger.setLevel(logging.WARN)

In [ ]:
cluster_index = 0
bin_size_ = 5

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(32, 16))

lc.render_2d(clust_selection.references[cluster_index], ax=ax, bin_size=bin_size_, rescale=lc.Trafo.EQUALIZE, cmap=m_gray.reversed())

locdatas_ = [clusts[0].references[cluster_index], clusts[1].references[cluster_index]]
lc.render_2d_rgb_mpl(locdatas_, bin_size=bin_size_, rescale=lc.Trafo.EQUALIZE);

ax.add_patch(clust_selection.references[cluster_index].convex_hull.region.as_artist(alpha=0.3))

ax.set(
    title=f'cluster {cluster_index}',
    aspect='equal',
    # xlim=(0, 2000),
    # ylim=(0, 2000)
)
ax.legend()
plt.show()

## Localizations property distributions for localizations in cluster selection

In [ ]:
locdata_clust_selection_channels = [lc.LocData.concat(clust_.references) for clust_ in clusts]

In [ ]:
columns = [column for column in locdata_clust_selection_channels[0].data.columns.to_list() if column not in 
           ['index', 'original_index', 'position_x', 'position_y', 'uncertainty_x', 'uncertainty_y', 'position_z', 'frame', 'channel']]
columns

In [ ]:
%%time
lprops = {}
for column in columns:
    for locdata_clust_selection in locdata_clust_selection_channels:
        lprops[column] = lc.LocalizationProperty(loc_property=column, index='frame').compute(locdata_clust_selection)

        if column == 'intensity':
            lprops[column].fit_distributions()
        else:
            lprops[column].fit_distributions(distribution=stats.norm)

        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        lprops[column].plot(window=1000, ax=axs[0])
        lprops[column].hist(ax=axs[1])
        plt.tight_layout()
        plt.show()

        print('Fit results: ', lprops[column].distribution_statistics.parameter_dict())

## Cluster properties

### Number of localizations per cluster

In [ ]:
clust_prop_nloc = [lc.LocalizationProperty(loc_property='localization_count').compute(clust_) for clust_ in clusts]
clust_prop_nloc

In [ ]:
for element in clust_prop_nloc:
    element.hist(bins=100, log=True, fit=False, alpha=0.5);

In [ ]:
for element in clust_prop_nloc:
    print(element.results.describe())

## Density per cluster

In [ ]:
localization_densities = [clust_.data.localization_density.values for clust_ in clusts]
for data in localization_densities:
    print(pd.Series(data).describe(), "\n")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 8))
ax.boxplot(localization_densities,
           labels=["0", "1"],
           showmeans=True);
ax.violinplot(localization_densities);